In [8]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from pathlib import Path
import sys
sys.path.append('..')
import src.config as cfg
import src.features as ft

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
df = pd.read_csv(Path('../data/FARS-data-full-sample.txt'), sep='\t')
# The replication does not use whether child seat or belt were improperly used, and thereafter drops rows with missing values in them
df_repl = df[[feat for feat in df.columns if not feat.startswith('imp')]]
df_repl = df_repl[~df_repl.isna().max(axis=1)]
df_repl['modelyr'] = df_repl['modelyr'].astype(int)
df.head()

,age,year,passgcar,suv,weekend,frimp,indfrimp,rearimp,indrearimp,rsimp,...,vehicles1,vehicles2,missweight,thoulbs_I,numcrash,drivebelt,crashcar,splmU55,lowviol,highviol
0,4,1975,1,0,1,0.0,1.0,0.0,0.0,0.0,...,0,0,0,4.633,6,0,1975100242,1.0,0.0,0.0
1,5,1975,1,0,0,1.0,0.0,0.0,0.0,0.0,...,0,1,0,3.863,5,0,1975101091,1.0,NaN,NaN
2,4,1975,1,0,1,0.0,1.0,0.0,0.0,0.0,...,0,1,0,3.784,6,0,1975101192,1.0,0.0,0.0
3,4,1975,1,0,0,0.0,1.0,0.0,0.0,0.0,...,0,1,1,0.000,3,0,1975102721,1.0,1.0,1.0
4,2,1975,1,0,1,0.0,1.0,0.0,0.0,0.0,...,0,1,0,3.330,3,0,1975102882,1.0,0.0,0.0


In [3]:
y = df_repl['death']
T = df_repl[['childseat', 'lapbelt', 'lapshould']].max(axis=1)
X = df_repl[['lapbelt', 'lapshould']]
W = df_repl[[col for col in df_repl.columns if col not in ['lapbelt', 'lapshould']]]

In [94]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor
from econml.causal_forest import CausalForestDML
from econml.dml import LinearDML, DML
from sklearn.model_selection import KFold

In [268]:
from sklearn.metrics import r2_score

In [52]:
from sklearn.linear_model import LinearRegression
controls = [item for typ in ft.feat_type.values() for item in typ]

results = []
pipe = ft.construct_data_pipeline(pd_output=True)
for feat in [col for col in controls if col in df_repl]: 
    dt = pipe.fit_transform(df_repl[[feat]])
    mdl = LinearRegression().fit(dt, y)
    results.append([feat, r2_score(y, mdl.predict(dt))])
ordered_features = pd.DataFrame(results, columns=['feature', 'r2']).sort_values('r2', ascending=False)['feature'].to_list()

In [9]:
y = df_repl['death']
T = df_repl['restraint']
W = df_repl[[col for col in df_repl.columns if col not in ['lapbelt', 'lapshould']]]

pipe = ft.construct_data_pipeline(pd_output=True)
cv_splitter = KFold(2, shuffle=True, random_state=cfg.random_state)

In [113]:
model_t = HistGradientBoostingClassifier(max_iter=100, n_iter_no_change=10, max_depth=10, early_stopping=True, random_state=cfg.random_state)
model_y = HistGradientBoostingRegressor(max_iter=100, n_iter_no_change=10, max_depth=10, early_stopping=True, random_state=cfg.random_state)
dml = LinearDML(model_t=model_t, model_y=model_y, cv=cv_splitter, discrete_treatment=True, fit_cate_intercept=True)

In [121]:
data = df.copy(deep=True)[~df['restraint'].isna()]
y = data['death']
# T = data[['childseat', 'lapbelt', 'lapshould']].max(axis=1)
T = data['restraint']
X = data[['lapbelt', 'lapshould']]
W = data[[col for col in data.columns if col not in ['lapbelt', 'lapshould']]]

In [122]:
dml.fit(y, T.values, W=pipe.fit_transform(data))

In [123]:
inf = dml.intercept__inference()
inf.summary_frame()

point_estimate  stderr   zstat  pvalue  \
X              T                                                              
cate_intercept T0_1_childseat                -0.154   0.005 -30.153     0.0   
               T0_2_lap only belt            -0.119   0.006 -20.388     0.0   
               T0_3_lap/should belt          -0.165   0.006 -28.396     0.0   

                                     ci_lower  ci_upper  
X              T                                         
cate_intercept T0_1_childseat          -0.162    -0.145  
               T0_2_lap only belt      -0.128    -0.109  
               T0_3_lap/should belt    -0.174    -0.155

In [124]:
model_t = HistGradientBoostingRegressor(max_iter=100, n_iter_no_change=10, max_depth=10, early_stopping=True, random_state=cfg.random_state)
model_y = HistGradientBoostingRegressor(max_iter=100, n_iter_no_change=10, max_depth=10, early_stopping=True, random_state=cfg.random_state)
dml = LinearDML(model_t=model_t, model_y=model_y, cv=cv_splitter, discrete_treatment=False, fit_cate_intercept=True)

In [127]:
data = df.copy(deep=True)[~df['restraint'].isna()]
y = data['death']
T = data[['childseat', 'lapbelt', 'lapshould']].max(axis=1)
# T = data['restraint']
X = data[['lapbelt', 'lapshould']]
W = data[[col for col in data.columns if col not in ['lapbelt', 'lapshould']]]

In [128]:
dml.fit(y, T.values, X=X, W=pipe.fit_transform(data))

In [131]:
dml.summary()

,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
lapbelt,-0.089,0.119,-0.742,0.458,-0.285,0.108
lapshould,0.09,0.106,0.853,0.394,-0.084,0.264
,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,-0.152,0.005,-28.175,0.0,-0.161,-0.143


In [132]:
from src.dmlutils import DML_diagnostics
diag = DML_diagnostics(cv_splitter)

In [133]:
print(diag.get_score_y(dml, r2_score, pipe.fit_transform(df_repl), y))

ValueError: X has 97 features but this estimator was trained with 351 features.

In [54]:
# diag.get_score_T(dml, r2_score, pipe.fit_transform(df_repl), T.str.split('_').str[0].astype(int))


0.08364475851550968


In [34]:
dml.model_final_._var
dml.model_final_.coef_

array([-0.14031197, -0.11424521, -0.15746388])

In [37]:
from src.dmlutils import DML_diagnostics
from sklearn.metrics import r2_score
diag = DML_diagnostics(cv_splitter)

0        0
2        0
3        0
4        0
5        0
        ..
53655    1
53656    1
53658    1
53662    0
53663    3
Name: restraint, Length: 48202, dtype: object

0.08364475851550968


In [106]:
import src.features as ft
filler = ft.NanFiller()
X = filler.fit_transform(df_repl)
print(df_repl.shape)

(48202, 34)


In [176]:
from sklearn.preprocessing import OneHotEncoder
test = df_repl[['suv', 'weekend']]

enc = OneHotEncoder(sparse=False)
enc.fit_transform(test)
enc.get_feature_names(test.columns)

array(['suv_0', 'suv_1', 'weekend_0', 'weekend_1'], dtype=object)

In [225]:
pipe = ft.construct_data_pipeline(pd_output=True)
pipe.fit_transform(df_repl)

['year_1975' 'year_1976' 'year_1977' 'year_1978' 'year_1979' 'year_1980'
 'year_1981' 'year_1982' 'year_1983' 'year_1984' 'year_1985' 'year_1986'
 'year_1987' 'year_1988' 'year_1989' 'year_1990' 'year_1991' 'year_1992'
 'year_1993' 'year_1994' 'year_1995' 'year_1996' 'year_1997' 'year_1998'
 'year_1999' 'year_2000' 'year_2001' 'year_2002' 'year_2003' 'year_2004'
 'year_2005' 'year_2006' 'year_2007' 'year_2008' 'year_2009'
 'modelyr_1970.0' 'modelyr_1971.0' 'modelyr_1972.0' 'modelyr_1973.0'
 'modelyr_1974.0' 'modelyr_1975.0' 'modelyr_1976.0' 'modelyr_1977.0'
 'modelyr_1978.0' 'modelyr_1979.0' 'modelyr_1980.0' 'modelyr_1981.0'
 'modelyr_1982.0' 'modelyr_1983.0' 'modelyr_1984.0' 'modelyr_1985.0'
 'modelyr_1986.0' 'modelyr_1987.0' 'modelyr_1988.0' 'modelyr_1989.0'
 'modelyr_1990.0' 'modelyr_1991.0' 'modelyr_1992.0' 'modelyr_1993.0'
 'modelyr_1994.0' 'modelyr_1995.0' 'modelyr_1996.0' 'modelyr_1997.0'
 'modelyr_1998.0' 'modelyr_1999.0' 'modelyr_2000.0' 'modelyr_2001.0'
 'modelyr_2002.0' 'mod

,year_1975,year_1976,year_1977,year_1978,year_1979,year_1980,year_1981,year_1982,year_1983,year_1984,...,row1,backright,backleft,backother,male,missweight,drivebelt,splmU55,lowviol,highviol
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0.0,0,0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0.0,0,0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1.0,1,0,1.0,1.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0.0,0,0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1.0,1,0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1.0,0,1,1.0,0.0,0.0
48198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1.0,0,1,1.0,1.0,0.0
48199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.0,0,1,1.0,0.0,0.0
48200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,1.0,0,1,0.0,0.0,0.0


In [190]:
import src.features as ft
hot = ft.OneHotPd(handle_unknown='ignore', sparse=False)
hot.fit_transform(test)

,suv_0,suv_1,weekend_0,weekend_1
0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0
2,1.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0
4,1.0,0.0,1.0,0.0
...,...,...,...,...
48197,0.0,1.0,0.0,1.0
48198,1.0,0.0,1.0,0.0
48199,1.0,0.0,1.0,0.0
48200,1.0,0.0,1.0,0.0


In [199]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
cat_pipeline = make_pipeline(ft.NanFiller(), ft.RareAggregator(), ft.OneHotPd(handle_unknown='ignore', sparse=False))
ordinal_pipeline = make_pipeline(ft.NanFiller())
ct = ColumnTransformer([
    ('cat_pipeline', cat_pipeline, get_feats(ft.feat_type['categorical']),
    ('ord_pipeline', ordinal_pipeline, ft.feat_type['ordinal'])
    ],
    remainder='passthrough'
)
ct.fit_transform(df_repl)

ValueError: A given column is not a column of the dataframe

In [168]:
test = df_repl[['suv', 'weekend', 'modelyr']]
recode = {}
for feat in test.columns:
    to_aggr = test[feat].value_counts()/test.shape[0] <= 0.01
    if to_aggr.sum()>0:
        recode[feat] = {val: 'other' if aggr else str(val) for val, aggr in dict(to_aggr).items()}
test.apply(lambda x: x.map(recode[x.name]) if x.name in recode.keys() else x)

,suv,weekend,modelyr
0,0,1,1975.0
2,0,1,1970.0
3,0,0,1971.0
4,0,1,1970.0
5,0,0,1973.0
...,...,...,...
53655,1,1,2005.0
53656,0,0,1996.0
53658,0,0,2005.0
53662,0,0,1999.0


In [64]:
(~df.isna().max(axis=1)).sum()

28240

In [56]:
df['crashtm'].value_counts()

1_day      44338
2_night     8523
3_morn      2528
Name: crashtm, dtype: int64